<a href="https://colab.research.google.com/github/srinihash/understandings_and_projects/blob/master/aws_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://realpython.com/python-boto3-aws-s3/

In [ ]:
!pip install boto3

In [3]:
# https://stackoverflow.com/questions/54571009/how-to-hide-secret-keys-in-google-colaboratory-from-users-having-the-sharing-lin
# Mounting my drive (do not worry, no one ca hack this because "login" is required for running this Colab)
!mkdir -p ~/.aws
!cp "/content/drive/My Drive/aws/config" ~/.aws/config

In [ ]:
# Client connection
import boto3
s3_client = boto3.client('s3')
print(dir(s3_client))

In [ ]:
# Direct Resource connection
import boto3
s3_resource = boto3.resource('s3')
print(dir(s3_resource))

boto3 is dependent upon lower library botocore : https://github.com/boto/botocore/tree/master

In [ ]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    bucket_name = create_bucket_name(bucket_prefix)
    bucket_response = s3_connection.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
        'LocationConstraint': current_region})
    print(bucket_name, current_region)
    return bucket_name, bucket_response

def create_temp_file(size, file_name, file_content):
    random_file_name = ''.join([str(uuid.uuid4().hex[:6]), file_name])
    with open(random_file_name, 'w') as f:
        f.write(str(file_content) * size)
    return random_file_name


first_bucket_name, first_response = create_bucket(bucket_prefix='firstpythonbucket',
                                                  s3_connection=s3_resource.meta.client)

second_bucket_name, second_response = create_bucket(bucket_prefix='secondpythonbucket',
                                                    s3_connection=s3_resource)

print("first bucket : {} :: second bucket : {}".format(first_bucket_name, 
                                                       second_bucket_name))

first_file_name = create_temp_file(300, 'firstfile.txt', 'f')

first_bucket = s3_resource.Bucket(name=first_bucket_name)
first_object = s3_resource.Object(bucket_name=first_bucket_name, 
                                  key=first_file_name)

Let's now upload an object to the created bucket, in this case a file !
We can upload the same using 3 constructs:
1. From a Object instance
2. From a Bucket instance
3. From the client

In [14]:
# Object instance version
s3_resource.Object(first_bucket_name, first_file_name).upload_file(Filename=first_file_name)

first_object.upload_file(first_file_name)

In [15]:
# Bucket instance version
s3_resource.Bucket(first_bucket_name).upload_file(Filename=first_file_name, Key=first_file_name) 

In [16]:
# Client version
s3_resource.meta.client.upload_file(Filename=first_file_name, Bucket=first_bucket_name, Key=first_file_name) 

Let's copy this Object from one bucket to another

In [21]:
def copy_to_bucket(bucket_from_name, bucket_to_name, file_name):
    copy_resource = {
        'Bucket': bucket_from_name,
        'Key': file_name 
    }
    s3_resource.Object(bucket_to_name, file_name).copy(copy_resource)

copy_to_bucket(first_bucket_name, second_bucket_name, first_file_name)

In [ ]:
s3_resource.Object(second_bucket_name, first_file_name).delete()

In [23]:
second_file_name = create_temp_file(400, 'secondfile.txt', 's')
second_object = s3_resource.Object(first_bucket_name, second_file_name)
second_object.upload_file(second_file_name, ExtraArgs={'ACL': 'public-read'})
second_object_acl = second_object.Acl()

In [ ]:
second_object_acl.grants

Encrypting your objects

In [26]:
third_file_name = create_temp_file(300, 'thirdfile.txt', 't')
third_object = s3_resource.Object(first_bucket_name, third_file_name)
third_object.upload_file(third_file_name, ExtraArgs={'ServerSideEncryption': 'AES256',
                                                     'ACL': 'public-read'})

In [ ]:
third_object.server_side_encryption

In [31]:
third_object.storage_class

Versioning your objects

In [ ]:
def enable_bucket_versioning(bucket_name):
    bkt_versioning = s3_resource.BucketVersioning(bucket_name)
    bkt_versioning.enable()
    print(bkt_versioning.status)

enable_bucket_versioning(first_bucket_name)

In [ ]:
first_bucket_name

In [36]:
s3_resource.Object(first_bucket_name, first_file_name).upload_file(first_file_name)
s3_resource.Object(first_bucket_name, first_file_name).upload_file(third_file_name)

In [ ]:
s3_resource.Object(first_bucket_name, first_file_name).version_id

Let's traverse contents of S3 of the account here 

In [ ]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

Traversing all contents of bucket

In [ ]:
for obj in first_bucket.objects.all():
    print(obj.key)

In [ ]:
for obj in first_bucket.objects.all():
    sub_rsc = obj.Object()
    print(obj.key, obj.storage_class, obj.last_modified, sub_rsc.version_id, sub_rsc.metadata)

Deleting Buckets and Objects

In [ ]:
def delete_all_objects(bucket_name):
    res = []
    bucket = s3_resource.Bucket(first_bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})

delete_all_objects(first_bucket_name)

In [ ]:
s3_resource.Bucket(first_bucket_name).delete()

In [ ]:
s3_resource.Bucket(second_bucket_name).delete()